In [ ]:
import torch
from src.models.crf import CRF
START_LABEL = "<START>"
STOP_LABEL = "<STOP>"
PAD_LABEL = "<PAD>"
LABEL_TO_IDX = {
    'A': 0,
    'B': 1,
    'C': 2,
    START_LABEL: 3,
    STOP_LABEL: 4,
    PAD_LABEL: 5,
}
K = len(LABEL_TO_IDX)
crf = CRF(LABEL_TO_IDX)

In [ ]:
# test calculate_sent_score
#def _cal_sent_score(self, feats, labels, masks):
# L, B = masks.shape
# assert feats.shape == (L, B, self.K)
# assert labels.shape == (L, B)
# score = torch.zeros(B, device=feats.device) # B
# labels = torch.cat([torch.full((1, B), self.label_to_idx[START_LABEL], dtype=torch.long, device=feats.device), labels], dim=0) # (L + 1) x B
# for t, feat in enumerate(feats):
#     score += (self.transitions[labels[t + 1], labels[t]] + feat[range(B), labels[t + 1]]) * masks[t] # (B + B) x B -> B
# # add the last transition to STOP_LABEL
# score += self.transitions[self.label_to_idx[STOP_LABEL], labels[torch.sum(masks, dim=0).long(), range(B)]]
# return score
feat = torch.arange(1, 1+K)
feats = torch.stack([torch.stack([feat, feat + 1]),
                      torch.stack([feat + 2, feat + 3]),
                    torch.stack([feat + 4, feat * 0])]) # L x B x K
labels = torch.tensor([[0, 0],
                    [1, 1],
                    [2, 5]]) # L x B
masks = torch.tensor([[1, 1],
                    [1, 1],
                    [1, 0]], dtype=torch.bool) # L x B
crf.transitions.data = torch.full_like(crf.transitions, -10000.)
crf.transitions.data[0:3, 0:3] = torch.tensor([[0, 1, 2],
                                        [3, 4, 5],
                                        [6, 7, 8]])
crf.transitions.data[LABEL_TO_IDX[STOP_LABEL], :] = feat
crf.transitions.data[:, LABEL_TO_IDX[START_LABEL]] = feat

In [ ]:
crf._cal_sent_score(feats, labels, masks) # B

In [ ]:
crf._cal_partition(feats, masks) # B

In [ ]:
def log_sum_exp(vec):
    B, K = vec.shape
    max_score = vec[range(B), torch.argmax(vec, dim=1)] # B
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score.view(-1, 1)), dim=1)) # B


In [1]:
%env http_proxy 127.0.0.1:7890
%env https_proxy 127.0.0.1:7890
from src.models.bilstm_crf import BiLSTMCRF
from src.variable import LABEL_TO_IDX
bilstm_crf = BiLSTMCRF(LABEL_TO_IDX, 1, 3)

env: http_proxy=127.0.0.1:7890
env: https_proxy=127.0.0.1:7890


/root/mambaforge/envs/bilstmcrf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import torch
word_ids = torch.tensor([[-1, 0, 1, 1, 1, -1],
                        [-1, 0, 1, 2, 2, -1]])
feat = torch.arange(1, 1+3, dtype=torch.float)
char_feats = torch.stack([torch.stack([feat, feat+1]),
                        torch.stack([feat+2, feat+3]),
                        torch.stack([feat+4, feat+5]),
                        torch.stack([feat+6, feat+7]),
                        torch.stack([feat+8, feat+9]),
                        torch.stack([feat+10, feat+11])]) # L x B x K
attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1],
                                [1, 1, 1, 1, 1, 1]], dtype=torch.bool) # L x B

In [11]:
bilstm_crf._char_feat_to_word_feat(char_feats, word_ids, attention_mask) # L x B x K

tensor([4, 4])
tensor([4, 4])


(tensor([[[ 3.,  4.,  5.],
          [ 4.,  5.,  6.]],
 
         [[ 7.,  8.,  9.],
          [ 6.,  7.,  8.]],
 
         [[ 0.,  0.,  0.],
          [ 9., 10., 11.]]]),
 tensor([[1., 1.],
         [1., 1.],
         [0., 1.]]))